### 5.6.1 面向对象游戏引擎

这个“游戏引擎”类中还包含窗口、精灵等对象，每个对象也都具有自己的数据属性和功能属性，比如窗口有长宽、标题、背景和前景颜色，可以在窗口的画布上绘制像素等，精灵也有自己的位置、图像、速度、运动等属性。“游戏引擎”和窗口、精灵之间是一种包含关系。
1.	游戏引擎类

```
其主要包含的属性有：
数据属性：
 游戏窗口（屏幕）window：窗口长宽、标题、绘制表面、背景或前景图像或颜色、字体颜色等
 所有的精灵Spirit 
 
功能属性：
初始化init()
    游戏主循环run()
         事件处理processEvent() 
         更新数据并处理碰撞collision()
         绘制场景render()
退出游戏quit()
```


In [1]:
import random
import pygame, sys
from pygame.locals import *

#----游戏状态常量--------
RUNNING = 1   #是否正在运行
PAUSE = -1    #暂停
STOP = 0      #停止

FPS  = 25  #画面刷新速率  

#----颜色 常量--------
BLACK = (0,0,0)
WHITE = (255, 255, 255)
GREEN = (0, 60, 0)
GREY = (210, 210 ,210)
RED = (255, 0, 0)
PURPLE = (255, 0, 255)
YELLOW = (120,120,0)

class GameEngine():
    #=========1. 初始化=============   
    def __init__(self, width, height,title = 'game Engine',background = None,foreground = None): 
        self.width, self.height,self.title = width, height,title
        pygame.init()       
        pygame.mixer.init()    #初始化播放声音引擎
        self.surface = pygame.display.set_mode((width, height))  
        pygame.display.set_caption(title)   #设置窗口标题
        self.runningState = RUNNING
        
        #------初始化背景或前景 background and foreground----
     #   self.background = background
     #   self.foreground = foreground
        self.set_background(background)                          
        self.set_foreground(foreground)  
        self.bg_color = BLACK    #背景颜色
        self.fg_color = YELLOW   #前景颜色
        self.logo_image = None
        
        #-------初始化游戏数据---------
        self.sprites_list = []       #初始化精灵数组 
        #...

#=========2. 游戏主循环===========
    def run(self):
        self.game_intro()
        clock = pygame.time.Clock()     #时钟        
        self.running = RUNNING
        while self.running != STOP:           
            self.running = self.processEvent()   # 2.1 事件处理
            if self.running == RUNNING:
                self.update()                # 2.2.1 更新数据
                self.collosion()              # 2.2.2 碰撞检测
                self.draw()                  # 2.3绘制场景
                pygame.display.flip()         # pygame.display.update()  
        
        clock.tick(FPS)                #每秒的帧频率FPS Frames
        pygame.quit()                   # 3. 退出程序
        
    def game_intro(self):
        pass      
                
    def set_foreground(self, image=None):
        self.foreground = image
        if image: 
            self.foreground = pygame.image.load(image)
         
    def set_background(self, image=None):
        self.background = image
        if image: 
            self.background = pygame.image.load(image)
    
    def init(self):        
        pass
    def add_sprite(self,sprite):
        self.sprites_list.append(sprite)
        
    def remove_sprite(self,sprite):
        self.sprites_list.remove(sprite)   
            
    #=======2.1 处理（键盘、鼠标等）事件===========    
    def processEvent(self):
        for event in pygame.event.get():     #返回当前的所有事件 
            if event.type == pygame.QUIT:  #接收到窗口关闭事件
                return STOP                #退出游戏 
            elif event.type == KEYDOWN:              
                return self.keydown(event)
            elif event.type == KEYUP:
                return self.keyup(event)
        return RUNNING                         #正常  
    
    #  键按下处理函数 keydown handler
    def keydown(self,event):
        if event.key == pygame.K_ESCAPE:
            return STOP     #退出
        return 1    #正常

    # 键弹起处理函数 keyup handler
    def keyup(self,event):
        return 1
        
    #===========2.2.1 更新数据================
    def update(self):
        for sprite in self.sprites_list:
            sprite.update()     
        for sprite in self.sprites_list:
            if sprite.is_dead():
                self.remove_sprite(sprite)    
    
    #===========2.2.2 碰撞检测处理================
    def collosion(self):
        pass
   
    #===========2.3 绘制场景=========
    def draw(self):        
        self.draw_background(self.surface)
        self.draw_foreground(self.surface)
        self.draw_sprites(self.surface) 
        self.draw_scores(self.surface) 
        #self.player.draw(self.surface)
 
    def draw_sprites(self, surface):       
        for sprite in self.sprites_list:
            sprite.draw(surface)       

    def draw_background(self, surface):
        surface.fill(self.bg_color)   # clear surface to green
        if self.background:
            surface.blit(self.background, (0,0))
            
    def draw_foreground(self, surface):
        if self.foreground:
            surface.blit(self.foreground, (0,0))   
 
    def draw_scores(self, surface):
        pass

    #在位置pos处用字体fontname和fontsize绘制文本txt
    def draw_text(self,text,pos,fontname = "Comic Sans MS", fontsize=20):
        #绘制得分 scores
        myfont1 = pygame.font.SysFont(fontname, fontsize)
        label1 = myfont1.render(text, 1, (255,255,0))
        self.surface.blit(label1, pos)  

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


### 2.	精灵

精灵是游戏中的主要角色，有各种各样的精灵（挡板、球、飞机、炮弹等），可以定义一个最基本的精灵类，表示所有不同种类精灵的共同特性，在此基础上再派生出特殊的精灵类。
精灵的共同属性有：（图片表示）的形象、位置、绘制（在屏幕上显示自己）、更新状态。

In [2]:
#=============所有精灵类的基类=================
class Sprite():
    def __init__(self, image=None, pos=(0,0),vel = (0,0)):
        self.pos = [pos[0],pos[1]]
        self.vel = [vel[0],vel[1]]       
        self.lives = 1
        self.set_image(image)                 
        
    #----更新状态---      
    def update(self):
        # 更新位置
        self.pos[0] += self.vel[0]
        self.pos[1] += self.vel[1]
       
        self.rect.x = int(self.pos[0]-self.rect.width//2)  
        self.rect.y = int(self.pos[1]-self.rect.height//2) 
        
    def draw(self, surface):
        if self.image:
            surface.blit(self.image, self.rect)  
        else:
            pygame.draw.rect(surface, WHITE, self.rect)    
        
    def  colliderect(self,sprite):
        return self.rect.colliderect(sprite.rect)
    
    def is_dead(self):
        return self.lives == 0
    
    def set_image(self,image):
        self.image = image
        if  image:
            self.image = pygame.image.load(image)
            image_rect = self.image.get_rect()            
            self.rect = pygame.rect.Rect(self.pos[0]-image_rect.width//2,
self.pos[1]-image_rect.height//2,
                                         image_rect.width,image_rect.height)   

### 5.6.2 Pong游戏

在上述Sprite类的基础上，可以定义Ball类表示乒乓球、Paddle类表示挡板：

In [3]:
class Ball(Sprite):
    def __init__(self, radius=15,pos=(0,0),image=None):
        super().__init__(image, pos) #调用父类的构造函数
        self.init_velocity()
        self.radius = radius
        if not image:
            self.rect = pygame.rect.Rect(pos[0]-radius, pos[1]-radius, 
                2*radius, 2*radius)
               
    def init_velocity(self):
        horizontal = random.randrange(12, 24)/30
        vertical = random.randrange(6, 18)/30
        if random.random()>0.5:   #改变水平速度方向
            horizontal= -horizontal
        if random.random()>0.5:   #改变垂直速度方向
            vertical= -vertical
        self.vel = [horizontal,-vertical]
        
    def draw(self, surface):
        if self.image:
            surface.blit(self.image, self.rect)
        else:
            pygame.draw.circle(surface, WHITE, (int(self.pos[0]),int(self.pos[1])), self.radius, 0)
            
     #----更新状态---        
    def update(self):
        super(Ball,self).update()
        
    
    def backward(self,horizonal = 1):
        if horizonal==1:
            self.vel[0] = - self.vel[0]* 1.1
        elif horizonal==-1:
            self.vel[1] = - self.vel[1]* 1.1
        else :  
            self.vel[0] = -self.vel[0]* 1.1
            self.vel[1] = -self.vel[1]* 1.1             

In [4]:
class Paddle(Sprite):
    def __init__(self,size=(8,80),pos=(0,0),image=None):
        super().__init__(image, pos)       
        self.size = [size[0],size[1]]
        if not image:
            self.rect = pygame.rect.Rect(pos[0]-size[0]//2, pos[1]-size[1]//2,
                                         size[0], size[1])       
            
    def draw(self, surface):
        if self.image:
            surface.blit(self.image, self.rect)
        else:
            GREEN = (0,255,0)
            pygame.draw.rect(surface, GREEN, (self.rect.x, self.rect.y,
                                             self.rect.width, self.rect.height))       


除了2个精灵类型Ball和Paddle，还需要在通用的游戏引擎类GameEngine基础上，派生定义一个表示Pong游戏的的游戏引擎类，假如叫PongGame。

In [7]:
PAD_MOVE_OFFSET = 8
class PongGame(GameEngine):
    def __init__(self, width, height,paddle_width=8,paddle_height=80,ball_radius = 15,title = 'Pong Game',\
                 background = None,foreground = None): 
        super().__init__( width, height,title,background,foreground)
        
        self.scores = [0,0]
        self.ball = Ball(ball_radius,[width//2, height//2])
        
        self.pad_width = paddle_width
        self.pad_height = paddle_height
        self.half_pad_width = paddle_width//2
        self.half_pad_height = paddle_height//2
        
        self.paddle1 = Paddle((self.pad_width,self.pad_height),(self.half_pad_width,height//2))
        self.paddle2 = Paddle((self.pad_width,self.pad_height),(width-self.half_pad_width,height//2))
        self.sprites_list = [self.ball,self.paddle1,self.paddle2]
        
        self.paddle_pressed = 0
   
    def collosion(self): 
        ball = self.sprites_list[0]
        paddle1 = self.sprites_list[1]
        paddle2 = self.sprites_list[2]
        
        width,height = self.surface.get_size()
        
        #上下墙碰撞，水平速度不变，垂直速度相反
        if ball.pos[1] < ball.radius or ball.pos[1] > height - 1 - ball.radius:
            ball.backward(-1)
            
        if ball.pos[0] < (ball.radius + paddle1.rect.width):
            if ball.pos[1] <= paddle1.pos[1] + self.half_pad_height and \
               ball.pos[1] >= paddle1.pos[1] - self.half_pad_height:
                    ball.backward(1)
                    self.scores[0] += 1
            else:
                ball.pos = [self.width//2,self.height//2]
                ball.init_velocity()
                self.scores[1] += 1
        
        if ball.pos[0] >  self.width - 1 - ball.radius - self.pad_width:           
            if ball.pos[1] <= paddle2.pos[1] + self.half_pad_height and  \
               ball.pos[1] >= paddle2.pos[1] - self.half_pad_height:
                    ball.backward(1)
                    self.scores[1] += 1
            else:
                ball.pos = [self.width//2,self.height//2]
                ball.init_velocity()
                self.scores[0] += 1              
    
        # 更新挡板的位置，保持挡板在窗口里
        if paddle1.pos[1] < self.half_pad_height or paddle1.pos[1]+self.half_pad_height> self.height:
            paddle1.pos[1] -=  paddle1.vel[1]
       
        if paddle2.pos[1] < self.half_pad_height or paddle2.pos[1]+self.half_pad_height> self.height:
            paddle2.pos[1] -=  paddle2.vel[1]
        
    # 按下按键的处理函数
    def keydown(self,event):
        if event.key == pygame.K_ESCAPE:  #退出
            return STOP    
        elif event.key == pygame.K_SPACE:  #暂停
            return PAUSE   
        elif event.key == K_UP:
            self.paddle2.vel[1] = -PAD_MOVE_OFFSET
            self.paddle_pressed = 2
        elif event.key == K_DOWN:
            self.paddle2.vel[1] = PAD_MOVE_OFFSET
            self.paddle_pressed = 2           
        elif event.key == K_w:
            self.paddle1.vel[1] = -PAD_MOVE_OFFSET
            self.paddle_pressed =1
        elif event.key == K_s:
            self.paddle1.vel[1] = PAD_MOVE_OFFSET 
            self.paddle_pressed = 1
        return RUNNING

    # 按键弹起的处理函数
    def keyup(self,event):
        if event.key in (K_w, K_s):
           self.paddle1.vel[1] = 0
        elif event.key in (K_UP, K_DOWN):
           self.paddle2.vel[1] = 0          
        return RUNNING

    #绘制得分
    def draw_scores(self, surface):
        self.draw_text(str(self.scores[0]),(10,10))
        self.draw_text(str(self.scores[1]),(self.width-30,10))


最后，可以用一个主程序，测试这个新的游戏PongGame：

In [9]:
#--------主程序--------
WIDTH = 640
HEIGHT = 480  
win = PongGame(WIDTH,HEIGHT)
win.run()
